In [1]:
'''
Process raw NCT00174655 records
to sequence patient records.
'''

!pip install sas7bdat
from sas7bdat import SAS7BDAT
from collections import defaultdict
import random
import numpy as np
from tqdm import tqdm
random.seed(1234)
np.random.seed(1234)
import dill
import os
os.chdir('./breast cancer/NCT00174655')

You should consider upgrading via the 'D:\python\python.exe -m pip install --upgrade pip' command.


In [2]:
'''
output patient record
baseline: a table
visit_day: visit days at each admission
visits: a patient record is like
{
    'order':['ae','..','..'] # the order of event types in each visit
    'visit':[
        [[0,1,2,],[0,1,2,3],[1,2,3]],  # first visit
        [[0,2,2],[0,2,3],[1,2,4]], # second visit
    ..],
}
vocab: {'ae': Voc}
'''

"\noutput patient record\nbaseline: a table\nvisit_day: visit days at each admission\nvisits: a patient record is like\n{\n    'order':['ae','..','..'] # the order of event types in each visit\n    'visit':[\n        [[0,1,2,],[0,1,2,3],[1,2,3]],  # first visit\n        [[0,2,2],[0,2,3],[1,2,4]], # second visit\n    ..],\n}\nvocab: {'ae': Voc}\n"

In [3]:
# structure each patient visit sequence
with SAS7BDAT('vis_date.sas7bdat', skip_header=False) as reader: # all visit information
    df = reader.to_data_frame()
df['ZVISNAME'].unique()
df = df[df['ZVISNAME'].map(lambda x: x not in ['ABBFU1','ABBFU2','SECMALG','ABBFU5','ABBFU6','ABBFU7','ABBFU8'])].reset_index(drop=True)
df['days'] = df[['ZVISD_B_DY','ZVISD_C_DY','ZVISD_F_DY','ZVISD_R_DY','ZVISD_SP_DY']].fillna(0).sum(1)
df_visit = df.copy()[['RUSUBJID','ZVISNAME','days']]
df_groupby = df[['ZVISNAME','days','RUSUBJID']].sort_values(by=['days']).groupby('RUSUBJID')
days_df = df_groupby['days'].agg(list).reset_index()
visit_df = df_groupby['ZVISNAME'].agg(list).reset_index()
visit_df = days_df.merge(visit_df,on='RUSUBJID')
visit_df['days'] = visit_df['days'].apply(lambda x: np.array(x)).apply(lambda x: (x - x[0]).tolist())
visit_df['days'] = visit_df['days'].map(lambda x: x[1:])
visit_df['ZVISNAME'] = visit_df['ZVISNAME'].map(lambda x: x[1:])
visit_df

,RUSUBJID,days,ZVISNAME
0,000315-000-999-001,"[7.0, 28.0, 49.0, 77.0, 105.0, 134.0, 168.0, 2...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
1,000315-000-999-002,"[2.0, 22.0, 42.0, 63.0, 84.0, 112.0, 140.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
2,000315-000-999-003,"[12.0, 33.0, 54.0, 75.0, 103.0, 138.0, 173.0, ...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
3,000315-000-999-004,"[6.0, 27.0, 48.0, 69.0, 90.0, 118.0, 146.0, 26...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
4,000315-000-999-005,"[1.0, 21.0, 59.0, 71.0, 80.0, 101.0, 129.0, 15...","[CHIMIO1, CHIMIO2, CHIMIO3, FOLUP2, CHIMIO4, C..."
...,...,...,...
987,000315-000-999-990,"[18.0, 40.0, 61.0, 82.0, 110.0, 138.0, 166.0, ...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
988,000315-000-999-991,"[7.0, 32.0, 53.0, 74.0, 93.0, 123.0, 151.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
989,000315-000-999-992,"[7.0, 35.0, 56.0, 84.0, 112.0, 147.0, 182.0]","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
990,000315-000-999-993,"[9.0, 29.0, 50.0, 78.0, 99.0, 134.0, 167.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."


In [4]:
df_visit = df_visit[~df_visit['ZVISNAME'].isin(['BASELINE'])].reset_index(drop=True)

In [5]:
# process ae events inside each visit
with SAS7BDAT('ae.sas7bdat', skip_header=False) as reader: # adverse effects
    df = reader.to_data_frame()
df['ZVISNAME'].unique()

array(['CHIMIO1', 'CHIMIO2', 'CHIMIO3', 'CHIMIO4', 'CHIMIO5', 'CHIMIO6',
       'CHIMIO7', 'FOLUP1', 'FOLUP2', 'FOLUP3', 'FOLUP4', 'BASELINE',
       'FOLUP5', 'CHIMIO8', 'CHIMIO9', 'FOLUP6', 'FOLUP7', 'FOLUP10',
       'FOLUP11', 'FOLUP8', 'FOLUP9', 'ABBFU5', 'ABBFU1'], dtype=object)

In [6]:
df = df.merge(df_visit, on=['ZVISNAME','RUSUBJID'], how='inner')
df = df.sort_values(by=['days'])
df = df[df['C_AECOS'].map(len) > 0].reset_index(drop=True)
# df['AECOS'] # COSTART adverse event coding system
# df['C_AECOS'] # Code of COSTART
df = df[['RUSUBJID','ZVISNAME','C_AECOS','AESER','days']]

df['AESER'] = df['AESER'].replace({'.':0,'No':0,'Yes':1})
df_ae = df.groupby(['RUSUBJID','ZVISNAME'])['C_AECOS'].unique().reset_index()
df['AESER'] = (df['AESER'] * df['C_AECOS']).replace({'':'NO'})
df_ae_ser = df.groupby(['RUSUBJID','ZVISNAME'])['AESER'].unique().reset_index()
df_ae_ser['AESER'] = df_ae_ser['AESER'].apply(lambda x: [x_ for x_ in x.tolist() if x_ != 'NO'] if len(x)>1 and 'NO' in x else x.tolist())
df_ae = df_ae.merge(df_ae_ser, on=['RUSUBJID','ZVISNAME'])
df_ae = df_ae.merge(df_visit, on=['ZVISNAME','RUSUBJID'], how='inner')
df_ae[df_ae['ZVISNAME'] == 'BCR1'] # no AE detected in BCR
df_ae

,RUSUBJID,ZVISNAME,C_AECOS,AESER,days
0,000315-000-999-001,CHIMIO1,"[00038, 00830, 01040, 00753, 01028, 00107]",[NO],6.0
1,000315-000-999-001,CHIMIO2,"[00038, 00753, 00307, 01028, 00836, 00997, 00695]",[NO],27.0
2,000315-000-999-001,CHIMIO3,"[00038, 00753, 00997, 00695, 01028, 00836]",[NO],48.0
3,000315-000-999-001,CHIMIO4,"[00695, 00447, 00997, 01028, 00038]",[NO],76.0
4,000315-000-999-001,CHIMIO5,"[01028, 00753, 00301, 00038, 00521]",[NO],104.0
...,...,...,...,...,...
8572,000315-000-999-994,CHIMIO6,"[00830, 01168, 00114, 00440, 00261, 00627, 011...","[01168, 00753]",120.0
8573,000315-000-999-994,CHIMIO7,"[00038, 00753, 00737, 00830, 01153, 00114, 006...",[NO],155.0
8574,000315-000-999-994,FOLUP1,"[00261, 00114, 00830, 00440, 00038, 01153, 00627]",[NO],287.0
8575,000315-000-999-994,FOLUP3,[01153],[NO],508.0


In [7]:
# with SAS7BDAT('bcrelap.sas7bdat', skip_header=False) as reader:  # 癌症复发
#     df = reader.to_data_frame()

In [8]:
# medication and dosage for the candidate treatment
with SAS7BDAT('ptxd.sas7bdat', skip_header=False) as reader: 
    df = reader.to_data_frame()
df = df[df['BTXCHOIX'].map(lambda x: x not in ['.'])].reset_index(drop=True)
df = df.sort_values(by=['D_OTXSTR_DY'])
df_groupby = df.groupby(by=['ZVISNAME','RUSUBJID'])
df_ptxd = df_groupby['BTXCHOIX'].unique().reset_index()
df_data = df_ptxd.merge(df_ae, on=['ZVISNAME','RUSUBJID'], how='outer')

In [9]:
df_data

,ZVISNAME,RUSUBJID,BTXCHOIX,C_AECOS,AESER,days
0,CHIMIO1,000315-000-999-001,"[DOXORUBICIN, CYCLOPHOSPHAMIDE]","[00038, 00830, 01040, 00753, 01028, 00107]",[NO],6.0
1,CHIMIO1,000315-000-999-002,[DOXORUBICIN],[00038],[NO],3.0
2,CHIMIO1,000315-000-999-003,[DOXORUBICIN],"[00038, 00249]",[NO],3.0
3,CHIMIO1,000315-000-999-004,"[DOXORUBICIN, CYCLOPHOSPHAMIDE]","[00038, 01168, 00429, 00831]",[NO],5.0
4,CHIMIO1,000315-000-999-005,"[CYCLOPHOSPHAMIDE, DOXORUBICIN]","[00038, 00753, 00114, 00718]",[NO],2.0
...,...,...,...,...,...,...
8662,FOLUP5,000315-000-999-993,NaN,[01153],[NO],638.0
8663,FOLUP6,000315-000-999-993,NaN,[01153],[NO],729.0
8664,FOLUP1,000315-000-999-994,NaN,"[00261, 00114, 00830, 00440, 00038, 01153, 00627]",[NO],287.0
8665,FOLUP3,000315-000-999-994,NaN,[01153],[NO],508.0


In [10]:
with SAS7BDAT('ptxd.sas7bdat', skip_header=False) as reader: 
    df = reader.to_data_frame()

In [11]:
# with SAS7BDAT('hospi.sas7bdat', skip_header=False) as reader: # 住院
#     df = reader.to_data_frame()
# df['ADMREAS'].unique() #住院原因
# df[df['ADMREAS'].map(lambda x: x != '.')]['ADMREAS'].value_counts()

In [12]:
# xray 超声等检查
# with SAS7BDAT('scan.sas7bdat', skip_header=False) as reader: 
#     df = reader.to_data_frame()
# df = df[['RUSUBJID','ZVISNAME','TYPE']]
# df = df[df['TYPE'].map(lambda x: x not in ['Not applicable due to prior Mastectomy','.'])].reset_index(drop=True)
# df_base = df[df['ZVISNAME']=='BASELINE']
# df = df[~df.index.isin(df_base.index)]
# df_groupby = df.groupby(by=['ZVISNAME','RUSUBJID'])
# df_scan = df_groupby['TYPE'].unique().reset_index()
# df_scan = df_scan.merge(df_visit, on=['RUSUBJID','ZVISNAME'], how='inner')
# df_data = df_data.merge(df_scan, how='outer')
# df_data
# df_base = df_base.groupby('RUSUBJID')['TYPE'].unique().reset_index()
# df_base = df_base.rename(columns={'TYPE':'scan_type'})
# df_base['scan_type'].map(len).value_counts()

In [13]:
# medication and dosage irrelevant to the candidate treatment
with SAS7BDAT('pctx.sas7bdat', skip_header=False) as reader: 
    df = reader.to_data_frame()
df = df.merge(df_visit)

# filter most common 100 med
df = df[df['C_CTXWHO'].map(len) > 1].reset_index(drop=True)
med_count = df.groupby(by=['C_CTXWHO']).size().reset_index().rename(columns={0:'count'}).sort_values(by=['count'], ascending=False).reset_index(drop=True)
df = df[df['C_CTXWHO'].isin(med_count.loc[:99,'C_CTXWHO'])]
df = df[['ZVISNAME','RUSUBJID','C_CTXWHO','days']].sort_values(by=['days'])
df_med = df.groupby(['RUSUBJID','ZVISNAME'])['C_CTXWHO'].unique().reset_index()
df_data = df_data.merge(df_med,how='outer')

In [14]:
with SAS7BDAT('vital.sas7bdat', skip_header=False) as reader:
    df = reader.to_data_frame()
df['ZVISNAME'].unique()
df_base = df[df['ZVISNAME'] == 'BASELINE']
df_vital = df[~df.index.isin(df_base.index)]
df_base = df_base[['RUSUBJID','PSKAR','HT','WT']].rename(columns={'PSKAR':'performance status', 'HT':'height','WT':'weight'})
df_vital = df_vital.dropna(subset=['PSKAR'])
df_vital = df_vital.merge(df_visit).sort_values(by='days')[['RUSUBJID','days','ZVISNAME','PSKAR']].reset_index(drop=True)

# df_data = df_data.merge(df_vital, on=['RUSUBJID','ZVISNAME','days'], how='left')

In [15]:
df_data

,ZVISNAME,RUSUBJID,BTXCHOIX,C_AECOS,AESER,days,C_CTXWHO
0,CHIMIO1,000315-000-999-001,"[DOXORUBICIN, CYCLOPHOSPHAMIDE]","[00038, 00830, 01040, 00753, 01028, 00107]",[NO],6.0,"[00955301001, 00016001001, 00041902003]"
1,CHIMIO1,000315-000-999-002,[DOXORUBICIN],[00038],[NO],3.0,"[00300101002, 00016001001, R0032201003, 002732..."
2,CHIMIO1,000315-000-999-003,[DOXORUBICIN],"[00038, 00249]",[NO],3.0,"[00955302R01, R0116901006, 00133002007, 000365..."
3,CHIMIO1,000315-000-999-004,"[DOXORUBICIN, CYCLOPHOSPHAMIDE]","[00038, 01168, 00429, 00831]",[NO],5.0,"[00661201001, 00955301001]"
4,CHIMIO1,000315-000-999-005,"[CYCLOPHOSPHAMIDE, DOXORUBICIN]","[00038, 00753, 00114, 00718]",[NO],2.0,"[00041902003, 00955302R01, 00016001001, 000365..."
...,...,...,...,...,...,...,...
9091,FOLUP5,000315-000-999-980,NaN,NaN,NaN,NaN,[00388701001]
9092,FOLUP1,000315-000-999-983,NaN,NaN,NaN,NaN,[00388701001]
9093,FOLUP5,000315-000-999-985,NaN,NaN,NaN,NaN,[00388701001]
9094,FOLUP1,000315-000-999-986,NaN,NaN,NaN,NaN,[00388701001]


In [16]:
# 抗癌治疗
# with SAS7BDAT('syscantx.sas7bdat', skip_header=False) as reader:
#     df = reader.to_data_frame()
# df['ZVISNAME'].value_counts()
# df = df[['ZVISNAME','RUSUBJID','C_CTXWHO','ZVISDATE_DY']].rename(columns={'ZVISDATE_DY':'days'})
# df = df[df['C_CTXWHO'].map(len) > 0].reset_index(drop=True)
# df_syscantx = df[df['ZVISNAME'].isin(['BCR1','BCR2'])]
# df_syscantx

In [17]:
# process death events or labels
with SAS7BDAT('death.sas7bdat', skip_header=False) as reader: # death events
    df = reader.to_data_frame()
df['ZVISNAME'].unique()
df_base = df_base.merge(df[['RUSUBJID','ZVISNAME']], how='outer')
df_base = df_base.rename(columns={'ZVISNAME':'death'})
df_base = df_base.fillna(0)

In [18]:
with SAS7BDAT('demog.sas7bdat', skip_header=False) as reader: # demographic
    df = reader.to_data_frame()
df = df[['AGE','RACE','SEX','RUSUBJID']]
df_base = df_base.merge(df, on=['RUSUBJID'], how='inner')

In [19]:
with SAS7BDAT('diag1.sas7bdat', skip_header=False) as reader: # 历史手术
    df = reader.to_data_frame()
print(df['ZVISNAME'].unique())
df_sg = df[['SURGERY','RUSUBJID']].groupby('RUSUBJID')['SURGERY'].unique().reset_index()
df_base = df_base.merge(df_sg, how='outer')
df_bcr = df_visit[df_visit['ZVISNAME'].isin(['BCR1','BCR2'])].reset_index(drop=True)
df_bcr = df_bcr.groupby('RUSUBJID')['ZVISNAME'].unique().reset_index()
df_base = df_base.merge(df_bcr, on=['RUSUBJID'],how='left')
df_base['num relapse']=df_base['ZVISNAME'].fillna('').map(len)
df_base = df_base.drop('ZVISNAME', axis=1)

['BASELINE']


In [20]:
with SAS7BDAT('diag2.sas7bdat', skip_header=False) as reader:
    df = reader.to_data_frame()
print(df['ZVISNAME'].unique())
df = df[['RUSUBJID','PATHOT','POSALN','PTSIZE','SITE_L','MULTUYN','HISGRADE','HISTYPE']]
df_1 = df[df['PTSIZE'].notnull()].reset_index(drop=True)[['RUSUBJID','PATHOT','PTSIZE','MULTUYN']]
df_2 = df[df['PTSIZE'].isnull()].reset_index(drop=True)[['RUSUBJID','POSALN','SITE_L','HISGRADE','HISTYPE']]
df_diag = df_1.merge(df_2, how='inner')
df_diag = df_diag.rename(columns={'PATHOT':'primary tumor',
                        'PTSIZE':'tumor size',
                        'MULTUYN':'multifocal tumor',
                        'POSALN':'num positive axillary lymph nodes',
                        'SITE_L':'tumor location',
                        'HISGRADE':'histopathologic grade',
                       'HISTYPE':'histopathologic type',})
df_diag['multifocal tumor'] = df_diag['multifocal tumor'].map(lambda x: 1 if x =='Yes' else 0)

['BASELINE']


In [21]:
df_base = df_base.merge(df_diag,on='RUSUBJID',how='left')

In [22]:
with SAS7BDAT('lab.sas7bdat', skip_header=False) as reader: # lab test results
    df = reader.to_data_frame()

In [23]:
df['LTEST'].value_counts()

Hemoglobin                18796
Neutrophils               18796
Platelets                 18796
White Blood Cells         18795
ASAT (SGOT)               12449
Alkaline Phosphatase      12448
ALAT (SGPT)               12448
Total Bilirubin           12447
Creatinine                 9717
Creatinine Clearance        997
                            936
Lutenising Hormone           63
Follicle Stim. Hormone       63
Name: LTEST, dtype: int64

In [24]:
# with SAS7BDAT('mx.sas7bdat', skip_header=False) as reader: # 疾病史 都在baseline
#     df = reader.to_data_frame()

In [25]:
# with SAS7BDAT('vd_cycle.sas7bdat', skip_header=False) as reader: 
#     df = reader.to_data_frame()
# df[df['RUSUBJID']=='000315-000-999-011']

In [26]:
# with SAS7BDAT('end.sas7bdat', skip_header=False) as reader:  # 停止化疗
#     df = reader.to_data_frame()
# # 停止化疗的原因
# df['ENDCHEMO'].value_counts()

In [27]:
os.chdir('../..')

In [28]:
visit_df

,RUSUBJID,days,ZVISNAME
0,000315-000-999-001,"[7.0, 28.0, 49.0, 77.0, 105.0, 134.0, 168.0, 2...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
1,000315-000-999-002,"[2.0, 22.0, 42.0, 63.0, 84.0, 112.0, 140.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
2,000315-000-999-003,"[12.0, 33.0, 54.0, 75.0, 103.0, 138.0, 173.0, ...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
3,000315-000-999-004,"[6.0, 27.0, 48.0, 69.0, 90.0, 118.0, 146.0, 26...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
4,000315-000-999-005,"[1.0, 21.0, 59.0, 71.0, 80.0, 101.0, 129.0, 15...","[CHIMIO1, CHIMIO2, CHIMIO3, FOLUP2, CHIMIO4, C..."
...,...,...,...
987,000315-000-999-990,"[18.0, 40.0, 61.0, 82.0, 110.0, 138.0, 166.0, ...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
988,000315-000-999-991,"[7.0, 32.0, 53.0, 74.0, 93.0, 123.0, 151.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
989,000315-000-999-992,"[7.0, 35.0, 56.0, 84.0, 112.0, 147.0, 182.0]","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."
990,000315-000-999-993,"[9.0, 29.0, 50.0, 78.0, 99.0, 134.0, 167.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ..."


In [29]:
df_data

,ZVISNAME,RUSUBJID,BTXCHOIX,C_AECOS,AESER,days,C_CTXWHO
0,CHIMIO1,000315-000-999-001,"[DOXORUBICIN, CYCLOPHOSPHAMIDE]","[00038, 00830, 01040, 00753, 01028, 00107]",[NO],6.0,"[00955301001, 00016001001, 00041902003]"
1,CHIMIO1,000315-000-999-002,[DOXORUBICIN],[00038],[NO],3.0,"[00300101002, 00016001001, R0032201003, 002732..."
2,CHIMIO1,000315-000-999-003,[DOXORUBICIN],"[00038, 00249]",[NO],3.0,"[00955302R01, R0116901006, 00133002007, 000365..."
3,CHIMIO1,000315-000-999-004,"[DOXORUBICIN, CYCLOPHOSPHAMIDE]","[00038, 01168, 00429, 00831]",[NO],5.0,"[00661201001, 00955301001]"
4,CHIMIO1,000315-000-999-005,"[CYCLOPHOSPHAMIDE, DOXORUBICIN]","[00038, 00753, 00114, 00718]",[NO],2.0,"[00041902003, 00955302R01, 00016001001, 000365..."
...,...,...,...,...,...,...,...
9091,FOLUP5,000315-000-999-980,NaN,NaN,NaN,NaN,[00388701001]
9092,FOLUP1,000315-000-999-983,NaN,NaN,NaN,NaN,[00388701001]
9093,FOLUP5,000315-000-999-985,NaN,NaN,NaN,NaN,[00388701001]
9094,FOLUP1,000315-000-999-986,NaN,NaN,NaN,NaN,[00388701001]


In [30]:
df_base['death'] = df_base['death'].apply(lambda x: 0 if x == 0 else 1)

In [31]:
df_base.to_csv('./NCT00174655/feature.csv',index=False)

def write_txt(x, fileloc):
    with open(fileloc, 'w', encoding='utf-8') as f:
        [f.write(l+'\n') for l in x]
        
binary_features = ['multifocal tumor']
write_txt(binary_features, './NCT00174655/binary_feature.txt')
numerical_features = ['performance status','height','weight','age','tumor size']
write_txt(numerical_features, './NCT00174655/numerical_feature.txt')

In [32]:
df_base

,RUSUBJID,performance status,height,weight,death,AGE,RACE,SEX,SURGERY,num relapse,primary tumor,tumor size,multifocal tumor,num positive axillary lymph nodes,tumor location,histopathologic grade,histopathologic type
0,000315-000-999-001,100.0,168.0,56.0,0,66.0,Caucasian,Female,"[Lumpectomy, Mastectomy]",0,Tumor 2cm <> 5cm in greatest dimension,3.4,0.0,1.0,Left,Poorly differentiated / Undifferentiated,Infiltrating ductal carcinoma
1,000315-000-999-002,100.0,149.0,66.0,0,44.0,Caucasian,Female,[Mastectomy],0,Tumor of <=2cm in its greatest dimension,1.5,1.0,1.0,Right,Moderately differentiated,Infiltrating ductal carcinoma
2,000315-000-999-003,100.0,170.0,70.3,0,41.0,Caucasian,Female,[Quadrantectomy/Segmental],0,Tumor 2cm <> 5cm in greatest dimension,2.2,0.0,1.0,Right,Poorly differentiated / Undifferentiated,Infiltrating ductal carcinoma
3,000315-000-999-004,100.0,141.0,81.0,0,61.0,Caucasian,Female,[Mastectomy],0,Tumor of <=2cm in its greatest dimension,2.0,0.0,4.0,Right,Poorly differentiated / Undifferentiated,Infiltrating ductal carcinoma
4,000315-000-999-005,100.0,154.0,57.9,0,51.0,Caucasian,Female,[Quadrantectomy/Segmental],0,Tumor 2cm <> 5cm in greatest dimension,4.7,0.0,4.0,Left,Poorly differentiated / Undifferentiated,Infiltrating ductal carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,000315-000-999-990,100.0,174.0,83.0,0,36.0,Caucasian,Female,[Quadrantectomy/Segmental],0,Tumor 2cm <> 5cm in greatest dimension,2.5,0.0,1.0,Right,Poorly differentiated / Undifferentiated,Other
990,000315-000-999-991,100.0,160.0,60.0,0,49.0,Caucasian,Female,[Quadrantectomy/Segmental],0,Tumor of <=2cm in its greatest dimension,1.6,0.0,6.0,Right,Well differentiated,Infiltrating lobular carcinoma
991,000315-000-999-992,90.0,159.0,61.0,0,38.0,Caucasian,Female,[Lumpectomy],0,Tumor of <=2cm in its greatest dimension,1.3,0.0,2.0,Right,Well differentiated,Infiltrating ductal carcinoma
992,000315-000-999-993,100.0,167.0,81.5,0,54.0,Caucasian,Female,[Lumpectomy],0,Tumor 2cm <> 5cm in greatest dimension,2.1,0.0,1.0,Right,Moderately differentiated,Infiltrating ductal carcinoma


In [33]:
df_data['RUSUBJID'].nunique()

977

In [34]:
# start to process sequence
df_data_key = df_data[['RUSUBJID','ZVISNAME']].drop_duplicates()
df_base_key = df_base[['RUSUBJID']].drop_duplicates()
df_visit_key = df_visit[['RUSUBJID','ZVISNAME']].drop_duplicates()
combined_key = df_visit_key.merge(df_data_key, on=['RUSUBJID','ZVISNAME'], how='inner')
combined_key = combined_key.merge(df_base_key, on=['RUSUBJID'], how='inner')

In [35]:
df_data = df_data.merge(combined_key, on=['RUSUBJID','ZVISNAME'], how='inner')
df_base = df_base.merge(combined_key['RUSUBJID'].drop_duplicates(), on=['RUSUBJID'], how='inner')
df_visit = df_visit.merge(combined_key, on=['RUSUBJID','ZVISNAME'], how='inner')

In [36]:
# visit_groupby = df_visit.sort_values(by='days').groupby('RUSUBJID')
# visit_df = visit_groupby['ZVISNAME'].agg(list).reset_index()
# days_df = visit_groupby['days'].agg(list).reset_index()
# visit_df = days_df.merge(visit_df)

In [37]:
# create vocabulary
class Voc(object):
    def __init__(self):
        self.idx2word = {0:'[PAD]'} # default padding index is 0
        self.word2idx = {'[PAD]':0}
    
    def __len__(self):
        return len(self.idx2word.keys())

    def add_sentence(self, sentence):
        for word in sentence:
            if word not in self.word2idx:
                self.idx2word[len(self.word2idx)] = word
                self.word2idx[word] = len(self.word2idx)

zvis_voc = Voc()
treat_voc = Voc()
ae_voc = Voc()
aeser_voc = Voc()
drug_voc = Voc()
v_stage_voc = Voc()
v_stage_voc.add_sentence(['treatment','followup'])

df_data.fillna('[PAD]', inplace=True)

for index, row in visit_df.iterrows():
    zvis_voc.add_sentence(row['ZVISNAME'])
    
for index, row in df_data.iterrows():
    if not len(row['BTXCHOIX'])==0:
        if isinstance(row['BTXCHOIX'], str):
            treat_voc.add_sentence([row['BTXCHOIX']])
        else:
            treat_voc.add_sentence(row['BTXCHOIX'])

    if not len(row['C_AECOS'])==0:
        if isinstance(row['C_AECOS'], str):
            ae_voc.add_sentence([row['C_AECOS']])
        else:
            ae_voc.add_sentence(row['C_AECOS'])
            
    if not len(row['C_CTXWHO'])==0:
        if isinstance(row['C_CTXWHO'], str):
            drug_voc.add_sentence([row['C_CTXWHO']])
        else:
            drug_voc.add_sentence(row['C_CTXWHO'])
            
    if isinstance(row['AESER'], str):
        aeser_voc.add_sentence([row['AESER']])
    else:
        aeser_voc.add_sentence(row['AESER'])

In [38]:
# create patient record
import pdb

# visit_df['timestamp'] = visit_df['days'].apply(lambda x: np.array(x)).apply(lambda x: (x - x[0]).tolist())
visit_df['timestamp'] = visit_df['days']
df_data = df_data.rename(columns={'BTXCHOIX':'treatment','C_AECOS':'adverse_event','AESER':'ae_serious','C_CTXWHO':'medication'})
records = []
visit_stages = []
timestamp = []
orders = ['treatment','medication','adverse_event','ae_serious','visit_stage','timestamp']
voc_dict = {
    'treatment':treat_voc,
    'medication': drug_voc,
    'adverse_event':ae_voc,
    'ae_serious':aeser_voc,
    'visit_stage':v_stage_voc,
    'timestamp':None,
}

map_event = lambda x: 0 if 'CHIMIO' in x else 1

subject_id_list = []
for subject_id in tqdm(df_base['RUSUBJID'].unique(), 'creating patient records'):
    subject_id_list.append(subject_id)
    item_df = df_data[df_data['RUSUBJID']==subject_id]
    item_visit = visit_df[visit_df['RUSUBJID']==subject_id]
    visit_stages.append([map_event(v) for v in item_visit['ZVISNAME'].tolist()[0]]) # 0: treatment phase, 1: followup phase    
    timestamp.append(item_visit['timestamp'].tolist()[0])
    patient = []
    for num_visit, (index, row) in enumerate(item_df.iterrows()):
        admission = []
        for order in orders:
            if order == 'visit_stage':
                admission.append(visit_stages[-1][num_visit])
            elif order =='timestamp':
                admission.append(timestamp[-1][num_visit])
            else:
                if isinstance(row[order], str): admission.append([voc_dict[order].word2idx[i] for i in [row[order]]])
                else: admission.append([voc_dict[order].word2idx[i] for i in row[order]]) 
        patient.append(admission)
    records.append(patient)
    

dill.dump(records, open('./NCT00174655/visit.pkl','wb'))
dill.dump(voc_dict, open('./NCT00174655/voc.pkl','wb'))
dill.dump(timestamp, open('./NCT00174655/timestamp.pkl','wb'))
dill.dump(visit_stages, open('./NCT00174655/visit_stage.pkl','wb'))

creating patient records: 100%|█████████████████████████████████████████████████████| 977/977 [00:01<00:00, 658.68it/s]


In [39]:
df_base.index = df_base['RUSUBJID']

In [40]:
df_base.loc[subject_id_list].reset_index(drop=True).to_csv('./NCT00174655/feature.csv',index=False)

In [41]:
visit_df

,RUSUBJID,days,ZVISNAME,timestamp
0,000315-000-999-001,"[7.0, 28.0, 49.0, 77.0, 105.0, 134.0, 168.0, 2...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ...","[7.0, 28.0, 49.0, 77.0, 105.0, 134.0, 168.0, 2..."
1,000315-000-999-002,"[2.0, 22.0, 42.0, 63.0, 84.0, 112.0, 140.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ...","[2.0, 22.0, 42.0, 63.0, 84.0, 112.0, 140.0, 28..."
2,000315-000-999-003,"[12.0, 33.0, 54.0, 75.0, 103.0, 138.0, 173.0, ...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ...","[12.0, 33.0, 54.0, 75.0, 103.0, 138.0, 173.0, ..."
3,000315-000-999-004,"[6.0, 27.0, 48.0, 69.0, 90.0, 118.0, 146.0, 26...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ...","[6.0, 27.0, 48.0, 69.0, 90.0, 118.0, 146.0, 26..."
4,000315-000-999-005,"[1.0, 21.0, 59.0, 71.0, 80.0, 101.0, 129.0, 15...","[CHIMIO1, CHIMIO2, CHIMIO3, FOLUP2, CHIMIO4, C...","[1.0, 21.0, 59.0, 71.0, 80.0, 101.0, 129.0, 15..."
...,...,...,...,...
987,000315-000-999-990,"[18.0, 40.0, 61.0, 82.0, 110.0, 138.0, 166.0, ...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ...","[18.0, 40.0, 61.0, 82.0, 110.0, 138.0, 166.0, ..."
988,000315-000-999-991,"[7.0, 32.0, 53.0, 74.0, 93.0, 123.0, 151.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ...","[7.0, 32.0, 53.0, 74.0, 93.0, 123.0, 151.0, 28..."
989,000315-000-999-992,"[7.0, 35.0, 56.0, 84.0, 112.0, 147.0, 182.0]","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ...","[7.0, 35.0, 56.0, 84.0, 112.0, 147.0, 182.0]"
990,000315-000-999-993,"[9.0, 29.0, 50.0, 78.0, 99.0, 134.0, 167.0, 28...","[CHIMIO1, CHIMIO2, CHIMIO3, CHIMIO4, CHIMIO5, ...","[9.0, 29.0, 50.0, 78.0, 99.0, 134.0, 167.0, 28..."


In [42]:
df_base

,RUSUBJID,performance status,height,weight,death,AGE,RACE,SEX,SURGERY,num relapse,primary tumor,tumor size,multifocal tumor,num positive axillary lymph nodes,tumor location,histopathologic grade,histopathologic type
RUSUBJID,,,,,,,,,,,,,,,,,
000315-000-999-001,000315-000-999-001,100.0,168.0,56.0,0,66.0,Caucasian,Female,"[Lumpectomy, Mastectomy]",0,Tumor 2cm <> 5cm in greatest dimension,3.4,0.0,1.0,Left,Poorly differentiated / Undifferentiated,Infiltrating ductal carcinoma
000315-000-999-002,000315-000-999-002,100.0,149.0,66.0,0,44.0,Caucasian,Female,[Mastectomy],0,Tumor of <=2cm in its greatest dimension,1.5,1.0,1.0,Right,Moderately differentiated,Infiltrating ductal carcinoma
000315-000-999-003,000315-000-999-003,100.0,170.0,70.3,0,41.0,Caucasian,Female,[Quadrantectomy/Segmental],0,Tumor 2cm <> 5cm in greatest dimension,2.2,0.0,1.0,Right,Poorly differentiated / Undifferentiated,Infiltrating ductal carcinoma
000315-000-999-004,000315-000-999-004,100.0,141.0,81.0,0,61.0,Caucasian,Female,[Mastectomy],0,Tumor of <=2cm in its greatest dimension,2.0,0.0,4.0,Right,Poorly differentiated / Undifferentiated,Infiltrating ductal carcinoma
000315-000-999-005,000315-000-999-005,100.0,154.0,57.9,0,51.0,Caucasian,Female,[Quadrantectomy/Segmental],0,Tumor 2cm <> 5cm in greatest dimension,4.7,0.0,4.0,Left,Poorly differentiated / Undifferentiated,Infiltrating ductal carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
000315-000-999-990,000315-000-999-990,100.0,174.0,83.0,0,36.0,Caucasian,Female,[Quadrantectomy/Segmental],0,Tumor 2cm <> 5cm in greatest dimension,2.5,0.0,1.0,Right,Poorly differentiated / Undifferentiated,Other
000315-000-999-991,000315-000-999-991,100.0,160.0,60.0,0,49.0,Caucasian,Female,[Quadrantectomy/Segmental],0,Tumor of <=2cm in its greatest dimension,1.6,0.0,6.0,Right,Well differentiated,Infiltrating lobular carcinoma
000315-000-999-992,000315-000-999-992,90.0,159.0,61.0,0,38.0,Caucasian,Female,[Lumpectomy],0,Tumor of <=2cm in its greatest dimension,1.3,0.0,2.0,Right,Well differentiated,Infiltrating ductal carcinoma


In [43]:
df_data['RUSUBJID'].nunique()

977

In [44]:
orders

['treatment',
 'medication',
 'adverse_event',
 'ae_serious',
 'visit_stage',
 'timestamp']

In [45]:
df_data[~df_data['ae_serious'].isin(['[PAD]',['NO']])].sort_values(by='RUSUBJID')

,ZVISNAME,RUSUBJID,treatment,adverse_event,ae_serious,days,medication
4802,CHIMIO6,000315-000-999-032,"[CYCLOPHOSPHAMIDE, METHOTREXATE, FLUOROURACIL]","[00753, 00114, 00619, 00209, 00038, 00307]",[00619],121.0,"[00041901001, 00020001003, 00116401003, R00229..."
41,CHIMIO1,000315-000-999-043,"[CYCLOPHOSPHAMIDE, DOXORUBICIN]","[00114, 01168, 00753, 00038]","[00114, 01168, 00753]",3.0,"[00041901002, 00008502003, 00614601003, R00322..."
1010,CHIMIO2,000315-000-999-043,"[DOXORUBICIN, CYCLOPHOSPHAMIDE]","[00038, 01168, 00114, 00622, 01173, 00831, 00753]","[01168, 00753]",27.0,"[00661201RI0, 00614601003, R0032201002, 000085..."
1976,CHIMIO3,000315-000-999-043,"[CYCLOPHOSPHAMIDE, DOXORUBICIN]","[00753, 00038, 00622, 00830, 00114, 00795, 011...",[00114],49.0,"[R0026301003, 00041901002, 00614601003, 000085..."
2930,CHIMIO4,000315-000-999-043,"[CYCLOPHOSPHAMIDE, DOXORUBICIN]","[00038, 00753, 01168, 00114, 00622, 01173]",[01168],70.0,"[00041901002, 00614601003, 00697201002, 006612..."
...,...,...,...,...,...,...,...
5672,CHIMIO6,000315-000-999-965,"[METHOTREXATE, FLUOROURACIL, CYCLOPHOSPHAMIDE]","[00038, 00751, 00830, 00622, 00084]","[00830, 00622]",119.0,"[00300101001, 00016201003, 00955301001]"
6584,CHIMIO7,000315-000-999-966,"[CYCLOPHOSPHAMIDE, FLUOROURACIL, METHOTREXATE]","[00362, 00327, 00107, 00737, 00114, 00751, 007...","[00362, 00327]",159.0,"[00041901001, 00955301001, 00016001001, R00263..."
8600,FOLUP1,000315-000-999-968,[PAD],"[00997, 00718]",[00997],249.0,[00388701001]
1922,CHIMIO2,000315-000-999-981,[DOXORUBICIN],"[01168, 00114, 00737, 00830, 00896, 00732, 009...",[00896],26.0,"[00016001001, 00139501002, 00239101001, 000149..."


In [46]:
len(voc_dict['ae_serious'])

77

In [47]:
voc_dict['ae_serious'].idx2word

{0: '[PAD]',
 1: 'NO',
 2: '00114',
 3: '01168',
 4: '00753',
 5: '00287',
 6: '00996',
 7: '00619',
 8: '00705',
 9: '00362',
 10: '01091',
 11: '00830',
 12: '00879',
 13: '00901',
 14: '00677',
 15: '00736',
 16: '00997',
 17: '01062',
 18: '01028',
 19: '00410',
 20: '00099',
 21: '00958',
 22: '00170',
 23: '00896',
 24: '00325',
 25: '00624',
 26: '00622',
 27: '00329',
 28: '01132',
 29: '00249',
 30: '00699',
 31: '00539',
 32: '00442',
 33: '00301',
 34: '00107',
 35: '00496',
 36: '00625',
 37: '00985',
 38: '00327',
 39: '00637',
 40: '00201',
 41: '01076',
 42: '00877',
 43: '00852',
 44: '00689',
 45: '00004',
 46: '00048',
 47: '00002',
 48: '00540',
 49: '00513',
 50: '00292',
 51: '00594',
 52: '00832',
 53: '00095',
 54: '00646',
 55: '00579',
 56: '00586',
 57: '00576',
 58: '00524',
 59: '00521',
 60: '00373',
 61: '00836',
 62: '00234',
 63: '00824',
 64: '00037',
 65: '00197',
 66: '00307',
 67: '01077',
 68: '00500',
 69: '00195',
 70: '00412',
 71: '00250',
 72: 